In [2]:
import pandas as pd
import altair as alt
import numpy as np
import re
import datetime as dt
import statsmodels.formula.api as sm
import statsmodels.formula.api as smf
class auto_car():
    """"
    A class provided an automation to search the best selling price of second hand car.
    ---
    Functions includ:
    transform: data transform base the informations of the car, eg model,brand,turbo, wrecked or not ,etc.
    clean: filter the outliers and ready to model
    prediction: provide a expect price of a car by linear regression
    visual: privide the optimisize price list on-selling records
    """

    def __init__(self,data):

        if isinstance(data,pd.DataFrame):
            self.data = data.drop_duplicates()
        else:
            raise TypeError("Input data is not valided")
        #self.transform()
        self.status = {"condition":False,"transform":False,"clean":False,"prediction":False,"rmse":None}

    

    def condition(self,brand = None,model=None, turbo = None,wrecked = None):
        """
        Conditions set-up
        """
        self._brand = brand
        self._model = model
        self._turbo = turbo
        self._wrecked = wrecked
        self.status["condition"] = True

    def model_information(self):
        if self.status["prediction"]:
            return print(self.best_model.summary() )
        else:
            try:
                self.prediction()
                return print(self.best_model.summary())
            except:
                return "Prediction error."

    def transform(self):
        """
        transform the dataframe by regular expression and extract those into a new column
        """

        pattern_price = r"\$\d+"
        pattern_kms = r"\d+"
        
        for i in self.data.index:
            #price
            tem = self.data.loc[i,"currentPrice"].replace(",","") 
            price = re.findall(pattern_price,tem)
            if price:
                self.data.loc[i,"Price"] = int(price[0].replace("$",""))
            else:
                self.data.loc[i,"Price"] = np.nan
                
            #previouse price
            tem = str(self.data.loc[i,"previousPrice"]).replace(",","") 
            p_price = re.findall(pattern_price,tem)

            if p_price:
                self.data.loc[i,"Previous_Price"] = int(p_price[0].replace("$",""))
            else:
                self.data.loc[i,"Previous_Price"] = np.nan

            if self.data.loc[i,"Previous_Price"] > 0:
                self.data.loc[i,"Price_Reduction"] = 1
                self.data.loc[i,"Reduced_price"] = self.data.loc[i,"Previous_Price"] - self.data.loc[i,"Price"]
            else:
                self.data.loc[i,"Price_Reduction"] = 0
                self.data.loc[i,"Reduced_price"] = np.nan
            


            #km
            tem = str(self.data.loc[i,"kms"]).lower()
            pattern_kms = r"\d+k\s"
            km = re.findall(pattern_kms,tem)
            if km:

                km[0] = km[0].replace("k","000")
                self.data.loc[i,"mileage/KM"] = int(km[0]) 

            else:
                self.data.loc[i,"mileage/KM"] = np.nan

            #year
            tem = str(self.data.loc[i,"name"]).lower().replace("-","")

            year_pattern = re.findall(r"[0-9]{4}",tem)
            if year_pattern:
                self.data.loc[i,"year"] = year_pattern[0]
            else:
                self.data.loc[i,"year"] = np.nan
            #self.data["year"] = self.data["year"] .astype(int)
            
            #brand
            brand = self._brand.lower()
            
            brand_pattern = re.findall(rf"{brand}",tem)
            if brand_pattern:
                self.data.loc[i,"brand"] = str(brand_pattern[0]).title()
            else:
                self.data.loc[i,"brand"] = np.nan

            #model
            model = self._model.lower().replace("-","")

            model_pattern = re.findall(rf"{model}",tem)
            if model_pattern:
                self.data.loc[i,"model"] = self._model
                #self.data.loc[i,"model"] = str(model_pattern[0]).title()
            #Turbo
            turbo_pattern = re.findall(r"\d\.\d",tem)
            if turbo_pattern:
                self.data.loc[i,"turbo"] = turbo_pattern[0]
            else:
                self.data.loc[i,"turbo"] = np.nan
            #wrecked or not
            wrecked_key_word = ["wreck"]
            wrecked = re.findall(r"wreck",tem)
            if wrecked:
                self.data.loc[i,"wrecked"] = 1
            else:
                self.data.loc[i,"wrecked"] = 0
        
        

        self.select_cols = ['Price', 'Previous_Price', 'Price_Reduction',"Reduced_price", 'mileage/KM', 'year',
       'brand', 'model', 'turbo', 'wrecked']
        
        self.status["transform"] = True

    def clean(self):
        """
        Elimited the outliers or extreme values and ready to model
        """
        if not self.status["transform"]:
            self.transform()

        self.predict_data = self.data.copy()
        df = self.predict_data 
        list_ = ['Price','mileage/KM']

        for col in list_:
            IQR = df[col].quantile(0.75) - df[col].quantile(0.25)
            lower_bound = df[col].quantile(0.25) - 1.5 * IQR
            upper_bound = df[col].quantile(0.75) + 1.5 * IQR
            df = df.loc[(df[col] >= lower_bound ) & (df[col] <= upper_bound)]
        year = dt.datetime.today().year

        df = df.loc[df['year'] <= str(year)]

        df["age"] = 2023 - df["year"].astype(int)

        if not self.status['condition'] :
            raise ValueError("Set up the brand and model first")
        df = df.loc[df["brand"] == self._brand]
        df = df.loc[df["model"] == self._model]
        self.predict_data = df

        self.status["clean"] = True

    def prediction(self):
        """
        Expected price prediction. Model choose from one of three models that depend by the best r^2.
        """
        
        if not self.status["clean"]  :
            self.clean()

        data =  self.predict_data

        #data['year']  = data['year'].astype(int)
        data['mileage'] = data['mileage/KM']
        
       
        model_linear = sm.ols("Price ~    mileage  +  year", data).fit()  # model 1
        model_square = sm.ols("Price ~    np.square(mileage) + year ", data).fit()   # model 2
        model_exp =  sm.ols("Price ~     np.exp(-mileage/1000) + year ", data).fit() # model 3

        self.model = {"model_linear":model_linear,"model_square":model_square,"model_exp":model_exp}

        # Select the best model
        best_model = self.model["model_linear"]
        best_r2 = self.model["model_linear"].rsquared

        for model in self.model.values():
            if model.rsquared > best_r2:
                best_model = model

        self.best_model = best_model

        data['prediction'] = best_model.predict()
        data['difference'] = round(data['prediction'],0) - data["Price"]
        self.output_data = data


        
        self.output_data = data
        self.status["prediction"]  = True

    def get_rmse(self):
        if self.status["prediction"]:
            rmse = 0
            for i in self.output_data.index:
                rmse += (self.output_data.loc[i,'difference'] ** 2)
            rmse = (rmse /self.output_data.shape[0]) ** 0.5  
            self.rmse = round(rmse,2) 
            return self.rmse 

    def visual(self,head = 10):
        if not self.status["prediction"]:
            self.prediction()
        
        self.output_data = self.output_data.sort_values(["difference","Reduced_price"],ascending=False)
        return self.output_data[["currentPrice","id","primaryPhotoURL","sellerLocation","sellerName", 'Previous_Price', 'Price_Reduction', 'Reduced_price',
       'mileage/KM', 'year', 'brand', 'model', 'turbo', 'wrecked',
       'prediction', 'difference']].head(head)


In [3]:

df = pd.read_excel("newconvertcsv.xlsx")
cx_5 = auto_car(df)
cx_5.condition(brand="Mazda",model="CX-5",turbo=None,wrecked=None)

print(cx_5.model_information(),'\n'*2,"="*20,"\n","Rmse: ",cx_5.get_rmse()) # All P-values present significantly

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.652
Model:                            OLS   Adj. R-squared:                  0.589
Method:                 Least Squares   F-statistic:                     10.31
Date:                Wed, 14 Jun 2023   Prob (F-statistic):           5.57e-08
Time:                        23:42:05   Log-Likelihood:                -495.59
No. Observations:                  53   AIC:                             1009.
Df Residuals:                      44   BIC:                             1027.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     2.414e+04   3882.634      6.218   

In [4]:
cx_5.visual()

,currentPrice,id,primaryPhotoURL,sellerLocation,sellerName,Previous_Price,Price_Reduction,Reduced_price,mileage/KM,year,brand,model,turbo,wrecked,prediction,difference
73,"AU$16,999",231516672919438,https://scontent.fadl7-1.fna.fbcdn.net/v/t39.3...,"Brisbane, Queensland, Australia",Nazir Amin,NaN,0.0,NaN,105000.0,2016,Mazda,CX-5,NaN,0.0,22908.595092,5910.0
81,"AU$23,500",770739451198097,https://scontent.fadl7-1.fna.fbcdn.net/v/t39.3...,"Brisbane, Queensland, Australia",Bilal Salehzada,NaN,0.0,NaN,63000.0,2017,Mazda,CX-5,NaN,0.0,27960.408004,4460.0
70,"AU$20,000",978307376874491,https://scontent.fadl7-2.fna.fbcdn.net/v/t39.3...,"Brisbane, Queensland, Australia",Joud T Karam,21999.0,1.0,1999.0,83000.0,2016,Mazda,CX-5,NaN,0.0,23744.526870,3745.0
69,"AU$21,500",172158982228931,https://scontent.fadl7-1.fna.fbcdn.net/v/t39.3...,"Toowoomba, Queensland",Dawod Hussein,23500.0,1.0,2000.0,54000.0,2016,Mazda,CX-5,NaN,0.0,24846.436942,3346.0
72,"AU$19,750",265610232605440,https://scontent.fadl7-2.fna.fbcdn.net/v/t39.3...,Rochedale South,Christina Lowry,NaN,0.0,NaN,103000.0,2016,Mazda,CX-5,NaN,0.0,22984.588890,3235.0
11,"AU$22,999",602406578600850,https://scontent.fadl7-2.fna.fbcdn.net/v/t39.3...,"Adelaide, South Australia",Arif Nawazi,24550.0,1.0,1551.0,110000.0,2017,Mazda,CX-5,NaN,0.0,26174.553750,3176.0
85,"AU$16,900",926310315490233,https://scontent.fadl7-1.fna.fbcdn.net/v/t39.3...,"Brisbane, Queensland, Australia",Nancy Tynan,17900.0,1.0,1000.0,66000.0,2015,Mazda,CX-5,NaN,0.0,19970.641072,3071.0
4,"AU$17,900",1377304826384635,https://scontent.fadl7-1.fna.fbcdn.net/v/t45.5...,"Adelaide, South Australia",Nick Bournias,NaN,0.0,NaN,162000.0,2016,Mazda,CX-5,NaN,0.0,20742.771849,2843.0
74,"AU$20,000",195810899546609,https://scontent.fadl7-1.fna.fbcdn.net/v/t39.3...,"Gold Coast, Queensland",Bruno Castro,22000.0,1.0,2000.0,107000.0,2016,Mazda,CX-5,NaN,0.0,22832.601294,2833.0
49,"AU$20,000",957839075262733,https://scontent.fadl7-2.fna.fbcdn.net/v/t39.3...,"Sydney, Australia",Stacey Veysi,NaN,0.0,NaN,108000.0,2016,Mazda,CX-5,NaN,0.0,22794.604395,2795.0


In [5]:
import pandas as pd
from ydata_profiling import ProfileReport
profile = ProfileReport(pd.read_csv('check1.csv'), explorative=True)

#Saving results to a HTML file
profile.to_file("output2.html")

FileNotFoundError: [Errno 2] No such file or directory: 'check1.csv'

In [ ]:
df = pd.read_excel("convertcsv.xlsx")
A4 = auto_car(df)
A4.condition(brand="Audi",model="A4",turbo=None,wrecked=None)
print(A4.model_information(),'\n'*2,"="*20,"\n","Rmse: ",A4.get_rmse()) 

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.786
Model:                            OLS   Adj. R-squared:                  0.745
Method:                 Least Squares   F-statistic:                     19.11
Date:                Mon, 12 Jun 2023   Prob (F-statistic):           4.21e-14
Time:                        21:44:55   Log-Likelihood:                -613.99
No. Observations:                  63   AIC:                             1250.
Df Residuals:                      52   BIC:                             1274.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                1

d:\Python\Python37\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
d:\Python\Python37\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
A4.visual()

,currentPrice,id,primaryPhotoURL,sellerLocation,sellerName,Previous_Price,Price_Reduction,Reduced_price,mileage/KM,year,brand,model,turbo,wrecked,prediction,difference
37,"AU$10,000",197924166070147,https://scontent.fadl7-1.fna.fbcdn.net/v/t39.3...,Denham Court,Nicholas Hilton,NaN,0.0,NaN,85000.0,2014,Audi,A4,NaN,0.0,21206.666677,11207.0
13,"AU$25,000",5462066450562943,https://scontent.fadl7-1.fna.fbcdn.net/v/t39.3...,"Melbourne, Victoria, Australia",Mohammad Baro,28000.0,1.0,3000.0,120000.0,2016,Audi,A4,NaN,0.0,33317.000000,8317.0
57,"AU$31,500",219630984195200,https://scontent.fadl7-2.fna.fbcdn.net/v/t39.3...,"Brisbane, Queensland, Australia",Shoaib Fazli,NaN,0.0,NaN,60000.0,2018,Audi,A4,NaN,0.0,39400.003806,7900.0
71,"AU$25,500",1005746770595229,https://scontent.fadl7-1.fna.fbcdn.net/v/t39.3...,"Gold Coast, Queensland",Rikki Turner,NaN,0.0,NaN,98000.0,2016,Audi,A4,NaN,0.0,33317.000000,7817.0
19,"AU$33,000",180800501568398,https://scontent.fadl7-1.fna.fbcdn.net/v/t39.3...,"Melbourne, Victoria, Australia",Dilip Reddy,NaN,0.0,NaN,113000.0,2018,Audi,A4,NaN,0.0,39400.003806,6400.0
0,"AU$27,990",148996304842358,https://scontent.fadl7-2.fna.fbcdn.net/v/t45.5...,"Adelaide, South Australia",David Cece,NaN,0.0,NaN,141000.0,2017,Audi,A4,2.0,0.0,34239.785714,6250.0
14,"AU$27,990",712330487311313,https://scontent.fadl7-2.fna.fbcdn.net/v/t45.5...,"Melbourne, Victoria, Australia",Motor Lane,NaN,0.0,NaN,56000.0,2017,Audi,A4,NaN,0.0,34239.785714,6250.0
16,"AU$28,000",3402736803387738,https://scontent.fadl7-1.fna.fbcdn.net/v/t39.3...,"Melbourne, Victoria, Australia",Aryan Bhallay,30000.0,1.0,2000.0,115000.0,2016,Audi,A4,NaN,0.0,33317.000000,5317.0
63,"AU$12,000",1321397955461681,https://scontent.fadl7-2.fna.fbcdn.net/v/t45.5...,"Gold Coast, Queensland",Fanis Shakirzianau,NaN,0.0,NaN,128000.0,2013,Audi,A4,NaN,0.0,16912.500000,4913.0
1,"AU$29,500",1475905089480955,https://scontent.fadl7-1.fna.fbcdn.net/v/t39.3...,"Adelaide, South Australia",Ahmed Al-Hasani,31800.0,1.0,2300.0,87000.0,2017,Audi,A4,NaN,0.0,34239.785714,4740.0


# Visualization

In [12]:
def extract_location_details(data):
    for row in data.index:
        tem = data.loc[row,"sellerLocation"].split(",")
        tem = [i.lstrip() for i in tem]
        if "Australia" in tem:
            tem.remove("Australia")
        if len(tem) == 2:
            data.loc[row,"stage"] = tem[1].strip()
        if tem[0] in ["Sydney","Minto Heights","Denham Court"]:
            data.loc[row,"stage"] = 'New South Wales'
        elif tem[0] in ["Meadowbrook","Rochedale South","Denham Court"]:
            data.loc[row,"stage"] = 'Queensland'
        data.loc[row,"city"] = tem[0]

        tem = data.loc[row,"name"].split(" ")

        for i in tem:
            i = i.lower()
            if i in ["cx-5","cx5","cx3","cx-3","mazda3","mazda2","cx-9","mazda6","bt-50"] or i in ["a3","a4","q5","q7","s3","q3","q2","a6","passat","s-line","golf","tiguan"]:
                data.loc[row,"model"] = i.upper()
    data["age"] = 2023 - data["year"]
                
def plot_stage_vs_model(stage,data=df):
    df = data.loc[data["stage"] == stage]
    plot= alt.Chart(df, title= f"Rank of frequancy of different model in {stage}").mark_bar().encode(
alt.Y( "model:N",axis=alt.Axis(labelAngle=0),sort=alt.EncodingSortField(
        op = "count",
        field = "Price",
        order = "descending")),
alt.X("count(Price):Q"),
alt.Size("average(Price):Q"),
alt.Color("brand:N"),
tooltip = ["count(Price)","average(Price)","median(Price)","max(Price)","min(Price)","average(age)"]
)
    
    return plot.interactive()


In [13]:
pd.set_option('display.max_rows', 500)
audi = pd.read_csv("audi.csv")
mazda = pd.read_csv("mazda.csv")
# alt.Title(
#        f"Median price of difference model in {stage}",
#        subtitle="All type of property"))

extract_location_details(audi)
extract_location_details(mazda)
audi_mazda = pd.concat([audi,mazda])
audi_mazda.loc[audi_mazda["model"].isna()] 

df = audi_mazda.loc[audi_mazda["Price"] > 1000]

stage = list(df["stage"].unique())

names = locals()

plot_dict = {}
for i in range(len(stage)):
    tem = str(stage[i]).replace(" ","")
    names[str(tem)] = plot_stage_vs_model(stage[i])
    plot_dict[i] = names[str(tem)]

plot_list = ""

for i in range(len(plot_dict)):
    plot_list += f"plot_dict[{i}]"

    if i < 4:
        if i % 2==0:

            plot_list += "&"
        else:
            plot_list += "|"


eval(plot_list)



alt.HConcatChart(...)